**Crawling MSW in wikipedia**

*   詞表：https://gist.github.com/indiejoseph/eae09c673460aa0b56db
*   libs: request/BS, Opencc (簡轉繁）, PageViews API

本來預計使用Wikipedia-API爬取（可看第一版本的function sense()）
後來發現他看不出href的位置、沒法爬gloss。

> 1st ver. WikiPedia-API ver: 1000 words: 2m19s

> 2nd ver. GetSenses: 1000 words: 3m左右

> 3rd ver. GetCoarseSense + GetSenses: 

Save MSW_dicts at 
`DIR = '/content/drive/MyDrive/110-1ComSem_final/SenseWdictv2'`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle 
import time 
import numpy as np
import requests
import json
import pandas as pd 
from bs4 import BeautifulSoup
path = '/content/drive/MyDrive/110-1ComSem_final/common_words_dict.pkl'
with open(path, 'rb') as f:
    cmdict = pickle.load(f)
#後來發現詞後面的那個數字不太像詞頻(問了github上的owner沒回），就先drop掉
cmwords = list(cmdict.keys())
cmwords[:10]

['阿爸', '阿昌族', '阿斗', '阿飛', '阿富汗', '阿訇', '阿拉伯數字', '阿拉伯語', '阿媽', '阿門']

In [ ]:
# 蘋果消歧義頁的條目
'''
苹果 (南韩电影): 苹果 (南韩电影) (id: ??, ns: 0)
苹果 (电影): 苹果 (电影) (id: ??, ns: 0)
苹果日报: 苹果日报 (id: ??, ns: 0)
蘋果公司: 蘋果公司 (id: ??, ns: 0)
蘋果唱片公司: 蘋果唱片公司 (id: ??, ns: 0)
蘋果園區: 蘋果園區 (id: ??, ns: 0)
蘋果日報 (臺灣): 蘋果日報 (臺灣) (id: ??, ns: 0)
蘋果日報 (香港): 蘋果日報 (香港) (id: ??, ns: 0)
蘋果日報慈善基金: 蘋果日報慈善基金 (id: ??, ns: 0)
黃暐婷: 黃暐婷 (id: ??, ns: 0)
'''

'\n苹果 (南韩电影): 苹果 (南韩电影) (id: ??, ns: 0)\n苹果 (电影): 苹果 (电影) (id: ??, ns: 0)\n苹果日报: 苹果日报 (id: ??, ns: 0)\n蘋果公司: 蘋果公司 (id: ??, ns: 0)\n蘋果唱片公司: 蘋果唱片公司 (id: ??, ns: 0)\n蘋果園區: 蘋果園區 (id: ??, ns: 0)\n蘋果日報 (臺灣): 蘋果日報 (臺灣) (id: ??, ns: 0)\n蘋果日報 (香港): 蘋果日報 (香港) (id: ??, ns: 0)\n蘋果日報慈善基金: 蘋果日報慈善基金 (id: ??, ns: 0)\n黃暐婷: 黃暐婷 (id: ??, ns: 0)\n'

### **Wikipedia API**

In [ ]:
!pip3 -q install Wikipedia-API
!pip install opencc-python-reimplemented
from opencc import OpenCC
cc = OpenCC('s2t')

     |████████████████████████████████| 484 kB 5.2 MB/s 
  Created wheel for opencc-python-reimplemented: filename=opencc_python_reimplemented-0.1.6-py2.py3-none-any.whl size=486150 sha256=d0457fb69195179c55fc12e8bff61f76d151c4939358a6550dc7c41f0dd4f517
  Stored in directory: /root/.cache/pip/wheels/4e/e2/60/d062d260be08788bb389521544a8fc173de9a9a78d6a593344
Successfully built opencc-python-reimplemented


In [ ]:
# !pip3 -q install Wikipedia-API
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('zh')
def printpage(page):
    if page.exists():
        print(f'Page Title: {page.title}\nPage Summary: {page.summary[:40]}...')
        return 
    
def disambig(page):
    '''Linear Search, links多的話runtime偏恐怖'''
    if not page.exists():
      return 
    links = page.links
    for title in links.keys():
        orig_title= title
        title = cc.convert(title).strip()
        match_title = cc.convert(page.title).strip()
        if match_title in title:  
            if "(消歧義)" in title:
              return {'wikipage':links[orig_title], 'url': f'https://zh.wikipedia.org/wiki/{orig_title}'}
    return 
#  CONs: can't get gloss; can only get entries with link 
'''
def senses(page):
    '抓取(消歧义)頁面的條目，只抓取有該詞彙在其中的條目（connotation或暱稱reference drop掉）12/12追加：使用(消歧义)頁面中的簡介作為gloss'
    sense_cands = []
    cands_views = []
    #### add main page as one sense #####
    #### 全以繁體辨識 #####
    target = page.title
    target = cc.convert(target) # 以下統一以繁體match
    target = target.rstrip('(消歧義)').strip()
    target_views = getViews(target)
    sense_cands.append(target)
    cands_views.append(target_views)
    links = page.links
    ###### get senses' names #####
    for title in links.keys():
        orig_title = title
        title = cc.convert(title)
        # (target != title)避免重複計入main sense
        if (target != title) and (target in title):
            # a sense candidate
            views = getViews(orig_title)
            cands_views.append(views)
            sense_cands.append(orig_title) # orig_title: 免得繁簡有差!
    ###### calculate Q1 ######
    # Q1 = np.percentile(cands_views, 25)
    # drop those below Q1 
    # but save the main page (the target page)
    # legal_cands = [(sense_cands[idx], views) for idx, views in enumerate(cands_views) if views > Q1]
    if (target,target_views) not in legal_cands:   
        legal_cands.append((target, target_views))
    return sorted(legal_cands, key = lambda x:x[1], reverse = True)
'''

"\ndef senses(page):\n    '抓取(消歧义)頁面的條目，只抓取有該詞彙在其中的條目（connotation或暱稱reference drop掉）12/12追加：使用(消歧义)頁面中的簡介作為gloss'\n    sense_cands = []\n    cands_views = []\n    #### add main page as one sense #####\n    #### 全以繁體辨識 #####\n    target = page.title\n    target = cc.convert(target) # 以下統一以繁體match\n    target = target.rstrip('(消歧義)').strip()\n    target_views = getViews(target)\n    sense_cands.append(target)\n    cands_views.append(target_views)\n    links = page.links\n    ###### get senses' names #####\n    for title in links.keys():\n        orig_title = title\n        title = cc.convert(title)\n        # (target != title)避免重複計入main sense\n        if (target != title) and (target in title):\n            # a sense candidate\n            views = getViews(orig_title)\n            cands_views.append(views)\n            sense_cands.append(orig_title) # orig_title: 免得繁簡有差!\n    ###### calculate Q1 ######\n    # Q1 = np.percentile(cands_views, 25)\n    # drop those below Q1 \n    # but save

### Crawling

In [ ]:
import requests
from bs4 import BeautifulSoup 
import re
import string

wiki_wiki = wikipediaapi.Wikipedia('zh')
def GetCoarseSense(fine_sense, disambig_page):
      '''Only works when the disambiguation page is properly sectioned'''
      coarse_senses = []
      for s in disambig_page.sections:
          coarse_sense = s.title
          # print(fine_sense,"-----", s.title)
          if (fine_sense in cc.convert(s.text)):
            return cc.convert(coarse_sense)
      return ''

def GetSenses(target, disambig_url, disambig_page):
    # combine with pageviews
    # disapple_url = 'https://zh.wikipedia.org/wiki/%E8%8B%B9%E6%9E%9C_(%E6%B6%88%E6%AD%A7%E4%B9%89)'
    page = requests.get(disambig_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    alllines = soup.find_all('li')
  
    senses = []
    Listed = []
    for line in alllines:
        # print(line)
        linkname = ''
        # 有外部link者
        atags = line.find_all('a')
        for atag in atags:
          # if link title is in atag: 
          # print(atag)
          try: 
              title = cc.convert(atag['title'])
              if target in title:
                  linkname = title
          except: 
              match = re.search(r'(?<=>)(((?!\<).)*)(?=\<\/a>)', str(line))
              linkname = match.group(0)
          if 'Special' in linkname or linkname in Listed:
              continue
          # get link for the sense    
          try: link = atag['href']
          except: link = ''
          # 處理linkname
          linkname = linkname.strip('（頁面不存在）')
          # print(linkname)
          t_linkname = cc.convert(linkname)
          # clean_linkname = ''.join([s for s in t_linkname if s not in string.punctuation])
          # clean tags 
          clean_line = re.sub(r'<.*?>', '', str(line))
          # replace newline
          clean_line = clean_line.replace('\n', '。')
          clean_line = re.sub(r'<.*?>', '', clean_line)
          # glsmatch = re.search(r'(?<=<\/a>)(((?!\<).)*)', line)
          gloss = clean_line.strip()
          if gloss == linkname:
              continue 
          # gloss用繁體
          gloss = cc.convert(gloss)
          if '消歧義' in gloss:
              continue

          # check coarse sense
          coarse_s = GetCoarseSense(linkname, disambig_page)
          views = getViews(linkname)
          Tuple = (coarse_s, linkname, views, gloss, link)
          senses.append(Tuple)
          Listed.append(linkname)
        
          if target in line: 
              print("target:", target, "line:", gloss)
              
              Tuple = ('', target, -1, gloss, '')
              senses.append(Tuple)
              Listed.append(linkname)
    # gloss裡面有target才留下
    senses = [sense for sense in senses if target in sense[-2]] # if target is inside gloss 

    return senses
# GetSenses("開心果", disambig_nut.fullurl)
# apple = wiki_wiki.page("城堡")
# disambig_apple_url = disambig_url(apple)
# Crawl(["城堡"])
# Crawl(["蘋果", "長城", '城堡', "冰", "藏語"])
# Crawl(["釣魚"])
# Crawl(["保安"])
# https://stackoverflow.com/questions/16780158/search-within-tags-with-beautifulsoup-python

In [ ]:
def getViews(pagename):
    # 3 years
    start = "20180101"
    end = "20211001"
    url = f'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/zh.wikipedia/all-access/all-agents/{pagename}/monthly/{start}/{end}'
    headers = {
    'User-Agent': 'My User Agent 1.0',
    'From': 'warrior290424@gmail.com' 
    }  # user agent for wiki's contact 
    response = requests.get(url, headers=headers)
    if "Not found." in response.text: 
       return -1
    dic = json.loads(response.text)
    viewsdata = pd.DataFrame.from_dict(dic)
    views = 0 
    for i in range(len(viewsdata)):
        views += viewsdata['items'][i]['views'] 
    return views 

# getViews("蘋果") # 34481
# getViews("蘋果公司") # 978087
# word = "蘋果"
# wordpage = wiki_wiki.page(word)
# (word, getViews(word), wordpage.summary[:40], wordpage.fullurl)

In [ ]:
def Crawl(cmwords = cmwords, verbose = True):
    SenseWdict = {}
    multisensewcount = 0
    for word in cmwords: # word必為繁體（已經事先轉好）
        word = word.strip()
        wordpage = wiki_wiki.page(word) # 簡繁皆可
        if wordpage.exists():
            # if verbose: printpage(wordpage)
            disa = disambig(wordpage)
            if disa:
                disambigurl = disa['url']
                disambigpage= disa['wikipage']
                time.sleep(0.0005)
                MS = GetSenses(word, disambigurl, disambigpage)
                # 寫入原條文
                if len(MS) >= 1:
                    # > 1 multi-sense 
                    summ = cc.convert(wordpage.summary[:50])
                    main = ('main', word, getViews(word), summ, wordpage.fullurl[24:])
                    MS.insert(0, main)
                    if verbose:
                        multisensewcount += 1 
                        print(f'Target {word} is crawled to have {len(MS)} sense(s).')
                    SenseWdict[word] = MS
    # finally 
    if verbose:
        print(f'{multisensewcount} words have been crawled to have multi-senses.')

    return SenseWdict  
# 1st ver. WikiPedia-API ver: 1000 words: 2m19s
# 2nd ver. GetSenses: 1000 words: 3m左右
# 3rd ver. GetCoarseSense + GetSenses: 

In [ ]:
!mkdir -p '/content/drive/MyDrive/110-1ComSem_final/SenseWdictv2'
DIR = '/content/drive/MyDrive/110-1ComSem_final/SenseWdictv2'

In [ ]:
from tqdm import tqdm 
step = 1000

start = time.time()
multisense_words = 0
cnt = 46
for x in tqdm(range(46000, len(cmwords), step)):
    cnt += 1
    sensewdict = Crawl(cmwords[x:(x+step)])
    multisense_words += len(sensewdict)
    with open(DIR+f'/MSWdict_{cnt}.json', 'w') as F:
      json.dump(sensewdict, F, indent = 4, ensure_ascii=False)
print("How many msw:", multisense_words)
end = time.time()
print(f"Runtime: {(end-start)//60} m {(end-start)%60} s")

  0%|          | 0/10 [00:00<?, ?it/s]

Target 玄武 is crawled to have 12 sense(s).
Target 學院 is crawled to have 3 sense(s).
Target 雪 is crawled to have 7 sense(s).
Target 雪豹 is crawled to have 6 sense(s).
Target 雪花 is crawled to have 5 sense(s).
Target 雪橇 is crawled to have 4 sense(s).
Target 旬 is crawled to have 4 sense(s).
Target 鴨 is crawled to have 4 sense(s).
Target 鴨子 is crawled to have 2 sense(s).
Target 雅思 is crawled to have 3 sense(s).
Target 煙 is crawled to have 3 sense(s).


 10%|█         | 1/10 [02:24<21:37, 144.19s/it]

Target 鹽田 is crawled to have 4 sense(s).
12 words have been crawled to have multi-senses.
Target 眼鏡蛇 is crawled to have 10 sense(s).
Target 演化 is crawled to have 7 sense(s).
Target 焰火 is crawled to have 2 sense(s).
Target 楊梅 is crawled to have 7 sense(s).
Target 洋 is crawled to have 5 sense(s).
Target 氧 is crawled to have 7 sense(s).
Target 要塞 is crawled to have 7 sense(s).
Target 野馬 is crawled to have 8 sense(s).
Target 葉 is crawled to have 6 sense(s).
Target 葉子 is crawled to have 5 sense(s).
Target 頁 is crawled to have 9 sense(s).
Target 夜叉 is crawled to have 4 sense(s).
Target 夜來香 is crawled to have 8 sense(s).
Target 夜曲 is crawled to have 8 sense(s).
Target 夜鶯 is crawled to have 3 sense(s).
Target 一 is crawled to have 3 sense(s).
Target 一見鍾情 is crawled to have 6 sense(s).


 20%|██        | 2/10 [05:00<20:10, 151.37s/it]

...

19 words have been crawled to have multi-senses.
Target 祝福 is crawled to have 15 sense(s).
Target 磚 is crawled to have 3 sense(s).
Target 資本主義 is crawled to have 4 sense(s).
Target 子彈 is crawled to have 9 sense(s).
Target 姊妹 is crawled to have 8 sense(s).
Target 紫羅蘭 is crawled to have 5 sense(s).
Target 字 is crawled to have 10 sense(s).
Target 字典 is crawled to have 3 sense(s).
Target 字體 is crawled to have 4 sense(s).
Target 字形 is crawled to have 3 sense(s).
Target 自然 is crawled to have 5 sense(s).
Target 自殺 is crawled to have 4 sense(s).


 90%|█████████ | 9/10 [22:24<02:29, 149.09s/it]

12 words have been crawled to have multi-senses.
Target 總督 is crawled to have 4 sense(s).
Target 足球 is crawled to have 11 sense(s).
Target 祖國 is crawled to have 11 sense(s).
Target 鑽石 is crawled to have 17 sense(s).


100%|██████████| 10/10 [24:09<00:00, 144.96s/it]

4 words have been crawled to have multi-senses.
How many msw: 122
Runtime: %24.0 m 9.580698490142822 s


In [ ]:
 line = '<li><a href="/wiki/%E5%85%83%E6%97%A6" title="元旦">元旦</a>：<a class="mw-redirect" href="/wiki/%E5%85%AC%E6%9B%86" title="公曆">公曆</a>新年</li>'
 line.strip('<li>').strip('</li>')

'a href="/wiki/%E5%85%83%E6%97%A6" title="元旦">元旦</a>：<a class="mw-redirect" href="/wiki/%E5%85%AC%E6%9B%86" title="公曆">公曆</a>新年'

In [ ]:
import pickle
import os
import json 
Big_wdict = {}
DIR = '/content/drive/MyDrive/110-1ComSem_final/SenseWdict'
for File in os.listdir(DIR):
    with open(DIR+f'/{File}', 'rb') as f:
        wdict = pickle.load(f)
    Big_wdict.update(wdict)
'''
with open(DIR+'/MSWdict.json', 'w') as F:
    json.dump(Big_wdict, F, ensure_ascii=False)
'''

"\nwith open(DIR+'/MSWdict.json', 'w') as F:\n    json.dump(Big_wdict, F, ensure_ascii=False)\n"

In [ ]:
len(Big_wdict) #2364
# self.word = word
# self.senses = [] # None 
stringWdict = {}
for msw in Big_wdict.values():
    if len(msw.senses)
    stringWdict[msw.word] = msw.senses

### issue1: 開心果現象
開心果：作為「果乾」，或作為「比喻一個人非常開心，同時也把這份喜悅傳給他人的人。」

然而維基詞條並未將後者列入連結（不太需要解釋），
反而列入了「果乾」、「綜藝節目條目」。
因此只爬連結（`<a>` tag）好像不太行。
但是就算了爬了「傳遞喜悅的人」的詞條，
也沒有data可供訓練。

Sol:
直接用request和BS爬，然後慢慢parse。
沒有連結的詞條也爬，就當作補全gloss。

### issue 2: 不知道PageViews metrics的效能/是否合理
之前計畫的heuristics是：calculate Q1
    # Q1 = np.percentile(cands_views, 25)
    # drop those below Q1 
    # but save the main page (the target page)
Sol(temp): 
現在：先不把sense剔除，但保留Pageviews的值在資料中供查詢。

In [ ]:
var = "開心果"
nut = wiki_wiki.page(var)
disambig_nut = disambig_page(nut) # https://zh.wikipedia.org/wiki/%E9%96%8B%E5%BF%83%E6%9E%9C_(%E6%B6%88%E6%AD%A7%E7%BE%A9)
parsed_nut = disambig_nut.text.split('\n')
print(*parsed_nut, sep = '\n')
Links = disambig_nut.links
Links 
var= '哭牆 (消歧義)'
page = wiki_wiki.page(var)
# page.fullurl
disambig_nut.fullurl

開心果可以指：

开心果：又被譯作「阿月渾子」，常見乾果一種。
「開心果」同時比喻一個人非常開心，同時也把這份喜悅傳給他人的人。
有「開心果」暱稱的已故香港藝人「肥姐」沈殿霞。
「開心『果』日」，原名為「水果日」，為香港衛生署的學界健康飲食盛會「健康飲食在校園」之一，在每年4月17日舉行，旨在提高全港學生的健康飲食意識。（註：該日子並非紀念沈殿霞，亦非鼓勵學生在該日子多吃開心果。加拿大溫哥華市政府為沈殿霞訂立紀念日「肥肥日」，在每年2月26日。）
開心果 (華視綜藝節目)：中華電視台（今中華電視公司）綜藝節目，共10集，播出期間為1987年4月19日至1987年6月28日每週日18:30－19:00，以知名歌星演出趣味短劇。
開心果 (中視綜藝節目)：中國電視公司綜藝節目，共6集，播出期間為1991年11月26日至1991年12月31日每週二17:30－18:00。
開心果 (台視綜藝節目)：台灣電視公司綜藝節目，共6集，播出期間為1992年1月7日至1992年2月18日每週二17:30－18:00。


'https://zh.wikipedia.org/wiki/%E9%96%8B%E5%BF%83%E6%9E%9C_(%E6%B6%88%E6%AD%A7%E7%BE%A9)'

### Testing **PageViews**

In [ ]:
#print(apple.fullurl)
# https://en.wikipedia.org/wiki/Python_(programming_language)
# print(apple.canonicalurl)
# https://en.wikipedia.org/wiki/Python_(programming_language)

In [ ]:
var = "蘋果"
url = f'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/zh.wikipedia/all-access/all-agents/{var}/monthly/20180101/20211001'
headers = {
    'User-Agent': 'My User Agent 1.0',
    'From': 'warrior290424@gmail.com'  # user agent for wiki's contact 
}
# YYYYMMDD
response = requests.get(url, headers=headers)
dic = json.loads(response.text)
viewsdata = pd.DataFrame.from_dict(dic)
# 要用 time.sleep(0.01) 
# Limit your clients to no more than 200 requests/s to this API.

In [ ]:
viewsdata['items'][0]

{'access': 'all-access',
 'agent': 'all-agents',
 'article': '蘋果',
 'granularity': 'monthly',
 'project': 'zh.wikipedia',
 'timestamp': '2018010100',
 'views': 1132}

In [ ]:
start = "20180101"
end = "20211001"
pagename = "安樂縣 (三帶州)"
pagename = "安樂縣 (清化州)"
url = f'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/zh.wikipedia/all-access/all-agents/{pagename}/monthly/{start}/{end}'
headers = {
'User-Agent': 'My User Agent 1.0',
'From': 'warrior290424@gmail.com' 
}  # user agent for wiki's contact 
response = requests.get(url, headers=headers)
getViews("安樂縣 (清化州)") # 978087

981